# Feature Engineering 2

### Load data

In [1]:
# Import packages
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Autoreload
%load_ext autoreload
%autoreload 2

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Import data
from bimbo.data import Bimbo

data = Bimbo().get_data()

# Load in the train and test datasets
train_df = data['train']
test_df = data['test']
cliente_tabla_df = data['cliente_tabla']
producto_tabla_df = data['producto_tabla']
town_state_df = data['town_state']
sample_submission_df = data['sample_submission']

Memory usage of dataframe is 0.04 MB
Memory usage after optimization is: 0.11 MB
Decreased by -184.7%
Memory usage of dataframe is 14.27 MB
Memory usage after optimization is: 19.51 MB
Decreased by -36.7%
Memory usage of dataframe is 373.80 MB
Memory usage after optimization is: 120.15 MB
Decreased by 67.9%
Memory usage of dataframe is 0.02 MB
Memory usage after optimization is: 0.02 MB
Decreased by 6.0%
Memory usage of dataframe is 6225.47 MB
Memory usage after optimization is: 2122.32 MB
Decreased by 65.9%
Memory usage of dataframe is 106.80 MB
Memory usage after optimization is: 33.38 MB
Decreased by 68.7%


### Create clusters of Products

In [2]:
products_id_clusters_df = pd.read_csv('../data/producto_clusters.csv')

In [3]:
products_id_clusters_df.head(3)

,Producto_ID,short_name,brand
0,0,NO IDENTIFICADO,IDENTIFICADO
1,9,Capuccino Moka,NES
2,41,Bimbollos Ext sAjonjoli,BIM


In [4]:
new_train_df = pd.merge(train_df, products_id_clusters_df, on='Producto_ID')

### Create a category of location based on zip code

In [5]:
town_state_df.head(3)

,Agencia_ID,Town,State
0,1110,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F."
2,1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO


In [6]:
town_state_df.nunique()

Agencia_ID    790
Town          260
State          33
dtype: int64

There are **33 States** in Mexico.

In [7]:
town_state_df['State'].unique().tolist()

['MÉXICO, D.F.',
 'ESTADO DE MÉXICO',
 'HIDALGO',
 'Queretaro de Arteaga',
 'PUEBLA',
 'OAXACA',
 'MORELOS',
 'GUERRERO',
 'TLAXCALA',
 'JALISCO',
 'COLIMA',
 'ZACATECAS',
 'NAYARIT',
 'SAN LUIS POTOSÍ',
 'AGUASCALIENTES',
 'MICHOACÁN',
 'TAMAULIPAS',
 'NUEVO LEÓN',
 'COAHUILA',
 'CHIHUAHUA',
 'DURANGO',
 'SONORA',
 'BAJA CALIFORNIA NORTE',
 'SINALOA',
 'BAJA CALIFORNIA SUR',
 'VERACRUZ',
 'GUANAJUATO',
 'QUERETARO',
 'TABASCO',
 'YUCATÁN',
 'CAMPECHE',
 'QUINTANA ROO',
 'CHIAPAS']

In [8]:
town_state_df['Town'].unique().tolist()

['2008 AG. LAGO FILT',
 '2002 AG. AZCAPOTZALCO',
 '2004 AG. CUAUTITLAN',
 '2029 AG.IZTAPALAPA 2',
 '2011 AG. SAN ANTONIO',
 '2001 AG. ATIZAPAN',
 '2007 AG. LA VILLA',
 '2013 AG. MEGA NAUCALPAN',
 '2018 AG. TEPALCATES 2',
 '2016 AG. SAN LORENZO',
 '2019 AG. XALOSTOC',
 '2094 CHALCO_BM',
 '2021 AG. XOCHIMILCO 2',
 '2017 AG. SANTA CLARA',
 '2003 AG. COACALCO',
 '2010 AG. LOS REYES',
 '2014 AG. NEZA',
 '2015 AG. ROJO GOMEZ',
 '2078 AG. TEXCOCO',
 '2088 AG. CEYLAN',
 '2087 AG. TIZAYUCA',
 '2089 AG. AZCAPOTZALCO INSTITUCIONALES',
 '2048 AG. IXTAPALUCA 1',
 '2116 Cruce De Anden Región Mexico',
 '2092 AG. SANTA CLARA AUTOSERVICIOS',
 '2059 Queretaro Balvanera',
 '2027 AG. VALLEJO',
 '2064 AG. TEPALCATES 1',
 '2041 AG. TULTITLAN',
 '2049 AG. IZTAPALAPA 1',
 '2050 AG. MIXCOAC',
 '2040 AG. CENTRO',
 '2042 AG. TEPOZOTLAN',
 '2070 AG. XOCHIMILCO 1',
 '2066 TOLUCA PONIENTE',
 '2056 PUEBLA NORTE MARINELA',
 '2057 PUEBLA SUR MARINELA',
 '2063 TEHUACAN MARINELA',
 '2053 OAXACA MARINELA',
 '2044 CUERNAV

In [9]:
new_train_df = pd.merge(new_train_df, town_state_df, on='Agencia_ID')

### Feature Engineering

In [10]:
new_new_train_df = new_train_df[new_train_df['Semana'] < 9]
val_df = new_train_df[new_train_df['Semana'] == 9]

In [12]:
from bimbo.preprocessing import *
from bimbo.feature_engineering import *

#### for the training/validation data

In [13]:
new_new_train_df = data_preprocess(new_new_train_df)
val_df = data_preprocess(val_df)

In [14]:
new_new_train_df = change_type_to_categ2(new_new_train_df)
val_df = change_type_to_categ2(val_df)

In [18]:
temp = feature_engineering2(new_new_train_df)

In [19]:
val = merge_feature2(val_df, temp, 'val')

In [20]:
val.to_csv('../data/processed_val_2.csv', index=False)

#### for the test data

In [21]:
real_train_df = data_preprocess(new_train_df)

In [22]:
real_train_df = change_type_to_categ2(real_train_df)

In [23]:
temp = feature_engineering2(real_train_df)

In [25]:
new_test_df = pd.merge(test_df, products_id_clusters_df, on='Producto_ID')
new_test_df = pd.merge(new_test_df, town_state_df, on='Agencia_ID')

In [26]:
test = merge_feature2(new_test_df, temp, 'test')

In [27]:
test.to_csv('../data/processed_test_2.csv', index=False)